In [1]:
import pandas as pd
import numpy as np


from core import constants
from core.utils import *

log = get_logger()

pd.set_option("display.float_format", "{:.2f}".format)

## RPGF 3 Data Check and Cleanup

In [2]:
df = pd.read_csv("data/dummy_data_rpgf3.csv")

In [3]:
display(df.sample(10))

,Address,Has voted,Has published,Published at,Created at,Updated at,Projects in ballot,Votes
794,WjXwaHf0lN,True,True,2023-11-29 03:09:24,2023-11-22 08:09:37,2023-11-28 14:15:08,18,"[{'amount': '668576', 'projectId': 'proj348'},..."
995,cPHi8aiAA5,True,False,NaN,2023-10-07 05:51:49,2023-10-19 07:49:45,0,[]
187,Nj001eEo8d,True,True,2023-11-30 12:17:07,2023-11-08 09:45:46,2023-11-29 11:36:10,17,"[{'amount': '694856', 'projectId': 'proj392'},..."
925,VPuqJPJx9K,True,False,NaN,2023-09-08 12:34:19,2023-11-02 11:58:30,0,[]
142,fHMrtFmYHy,False,False,NaN,2023-10-12 03:00:06,2023-11-20 04:22:33,0,[]
53,r86ZfnN8du,False,False,NaN,2023-09-06 21:05:30,2023-10-15 12:49:27,0,[]
564,2valr4ATwV,True,True,2023-10-20 01:00:30,2023-09-01 13:03:16,2023-10-01 19:41:43,7,"[{'amount': '492461', 'projectId': 'proj355'},..."
795,QOau96Qzje,True,False,NaN,2023-09-09 08:50:06,2023-10-17 13:25:21,0,[]
678,W7Et2mYl5L,True,True,2023-11-11 22:20:16,2023-10-14 15:22:32,2023-11-02 13:15:54,9,"[{'amount': '664821', 'projectId': 'proj364'},..."
546,BFK6gsvI3J,False,False,NaN,2023-10-07 20:00:24,2023-11-07 22:40:43,0,[]


In [4]:
log.info("Check - Num Ballots: " + str(df["Has published"].count()))
log.info("Check - Num Submissions (Published): " + str(df["Has published"].sum()))

# Check if voter_address is unique
if df["Address"].nunique() == df.shape[0]:
    log.info("Check - Address is unique.")
else:
    diff = df.shape[0] - df["Address"].nunique()
    log.info(f"Check - Address is not unique. There are {diff} duplicates.")

# Check if all voters have voted
if df[df["Has voted"] == False].shape[0] > 0:
    not_voted = df[df["Has voted"] == False].shape[0]
    total = df["Address"].nunique()
    log.info(f"Check - {not_voted} voters out of {total} have not voted.")
else:
    log.info("Check - All voters have voted.")

# Check if all voters have published
if df[df["Has published"] == False].shape[0] > 0:
    not_voted = df[df["Has published"] == False].shape[0]
    total = df["Address"].nunique()
    log.info(f"Check - {not_voted} voters out of {total} have not published.")
else:
    log.info("Check - All voters have published.")

2024-01-02 19:52:36 INFO | Check - Num Ballots: 1000
2024-01-02 19:52:36 INFO | Check - Num Submissions (Published): 514
2024-01-02 19:52:36 INFO | Check - Address is unique.
2024-01-02 19:52:36 INFO | Check - 243 voters out of 1000 have not voted.
2024-01-02 19:52:36 INFO | Check - 486 voters out of 1000 have not published.


In [5]:
# Apply the function and concatenate results
expanded_list = [
    expand_json(safe_json_loads(row), idx) for idx, row in df["Votes"].items()
]
expanded_df = pd.concat(expanded_list, ignore_index=True)

result_df = expanded_df.set_index("original_index").join(df.set_index(df.index))

In [6]:
testing_address = "zgdSu8Yr87"
print_df = result_df[result_df["Address"] == testing_address]
print("Num Projects Voted : " + str(print_df["projectId"].count()))
display(print_df.head(10))

Num Projects Voted : 18


,amount,projectId,Address,Has voted,Has published,Published at,Created at,Updated at,Projects in ballot,Votes
830,703749,proj117,zgdSu8Yr87,True,True,2023-11-02 18:18:29,2023-09-21 02:51:22,2023-09-23 16:56:39,18,"[{'amount': '703749', 'projectId': 'proj117'},..."
830,385034,proj51,zgdSu8Yr87,True,True,2023-11-02 18:18:29,2023-09-21 02:51:22,2023-09-23 16:56:39,18,"[{'amount': '703749', 'projectId': 'proj117'},..."
830,470511,proj230,zgdSu8Yr87,True,True,2023-11-02 18:18:29,2023-09-21 02:51:22,2023-09-23 16:56:39,18,"[{'amount': '703749', 'projectId': 'proj117'},..."
830,113619,proj204,zgdSu8Yr87,True,True,2023-11-02 18:18:29,2023-09-21 02:51:22,2023-09-23 16:56:39,18,"[{'amount': '703749', 'projectId': 'proj117'},..."
830,156327,proj141,zgdSu8Yr87,True,True,2023-11-02 18:18:29,2023-09-21 02:51:22,2023-09-23 16:56:39,18,"[{'amount': '703749', 'projectId': 'proj117'},..."
830,622540,proj390,zgdSu8Yr87,True,True,2023-11-02 18:18:29,2023-09-21 02:51:22,2023-09-23 16:56:39,18,"[{'amount': '703749', 'projectId': 'proj117'},..."
830,32962,proj482,zgdSu8Yr87,True,True,2023-11-02 18:18:29,2023-09-21 02:51:22,2023-09-23 16:56:39,18,"[{'amount': '703749', 'projectId': 'proj117'},..."
830,911617,proj46,zgdSu8Yr87,True,True,2023-11-02 18:18:29,2023-09-21 02:51:22,2023-09-23 16:56:39,18,"[{'amount': '703749', 'projectId': 'proj117'},..."
830,755757,proj250,zgdSu8Yr87,True,True,2023-11-02 18:18:29,2023-09-21 02:51:22,2023-09-23 16:56:39,18,"[{'amount': '703749', 'projectId': 'proj117'},..."
830,114436,proj82,zgdSu8Yr87,True,True,2023-11-02 18:18:29,2023-09-21 02:51:22,2023-09-23 16:56:39,18,"[{'amount': '703749', 'projectId': 'proj117'},..."


In [7]:
columns = [col for col in result_df.columns if col not in ["amount", "projectId"]]
columns += ["amount", "projectId"]  # Add the columns to the end of the list
result_df = result_df[columns]

# Update df columns names
result_df.columns = [
    "voter_address",
    "has_voted",
    "has_published",
    "published_at",
    "created_at",
    "updated_at",
    "projects_in_ballot",
    "votes",
    "amount",
    "project_id",
]

result_df.drop(columns="votes", inplace=True)

result_df["amount"] = pd.to_numeric(result_df["amount"])

In [8]:
# result_df.head()
result_df[result_df["voter_address"] == testing_address].head(70)

,voter_address,has_voted,has_published,published_at,created_at,updated_at,projects_in_ballot,amount,project_id
830,zgdSu8Yr87,True,True,2023-11-02 18:18:29,2023-09-21 02:51:22,2023-09-23 16:56:39,18,703749,proj117
830,zgdSu8Yr87,True,True,2023-11-02 18:18:29,2023-09-21 02:51:22,2023-09-23 16:56:39,18,385034,proj51
830,zgdSu8Yr87,True,True,2023-11-02 18:18:29,2023-09-21 02:51:22,2023-09-23 16:56:39,18,470511,proj230
830,zgdSu8Yr87,True,True,2023-11-02 18:18:29,2023-09-21 02:51:22,2023-09-23 16:56:39,18,113619,proj204
830,zgdSu8Yr87,True,True,2023-11-02 18:18:29,2023-09-21 02:51:22,2023-09-23 16:56:39,18,156327,proj141
830,zgdSu8Yr87,True,True,2023-11-02 18:18:29,2023-09-21 02:51:22,2023-09-23 16:56:39,18,622540,proj390
830,zgdSu8Yr87,True,True,2023-11-02 18:18:29,2023-09-21 02:51:22,2023-09-23 16:56:39,18,32962,proj482
830,zgdSu8Yr87,True,True,2023-11-02 18:18:29,2023-09-21 02:51:22,2023-09-23 16:56:39,18,911617,proj46
830,zgdSu8Yr87,True,True,2023-11-02 18:18:29,2023-09-21 02:51:22,2023-09-23 16:56:39,18,755757,proj250
830,zgdSu8Yr87,True,True,2023-11-02 18:18:29,2023-09-21 02:51:22,2023-09-23 16:56:39,18,114436,proj82


## Calculate Voting Results

In [9]:
allocator = ProjectAllocator(
    total_amount=constants.TOTAL_AMOUNT,
    min_amount=constants.MIN_AMOUNT,
    quorum=constants.QUORUM,
)

In [10]:
initial_allocation = allocator.calculate_initial_allocation(result_df)

In [11]:
display(initial_allocation.sample(10))

,votes_count,median_amount,is_eligible
project_id,,,
proj137,13,397278.00,False
proj444,8,556807.00,False
proj83,14,490712.00,False
proj379,9,427282.00,False
proj280,8,520589.50,False
proj383,13,472345.00,False
proj367,9,370685.00,False
proj344,12,454298.50,False
proj11,13,537648.00,False


In [12]:
# Scaling the total to 30M OP by project and filter out those with < 1500 OP
allocation_iter = initial_allocation[initial_allocation["is_eligible"] == True].copy()
allocation_iter["scaled_amount"] = allocation_iter["median_amount"]
# display(allocation_iter)
# Set a maximum number of iterations to prevent infinite loop
max_iterations = 10
current_iteration = 0

while (
    allocation_iter["scaled_amount"].sum() != constants.TOTAL_AMOUNT
    and current_iteration <= max_iterations
):
    allocation_iter = allocator.scale_allocations_oneby(allocation_iter)
    current_iteration += 1

    log.info("Check - Current iteration: " + str(current_iteration))

2024-01-02 19:52:36 INFO | Check - Original Amount Eligible: 12322787.0
2024-01-02 19:52:36 INFO | Check - Scale Factor: 2.434514205268662
2024-01-02 19:52:36 INFO | Check - No projects below minimum OP
2024-01-02 19:52:36 INFO | Check - Current iteration: 1


In [13]:
# Check if the loop exited due to reaching max iterations
if (
    current_iteration == max_iterations
    and allocation_iter["scaled_amount"].sum() != constants.TOTAL_AMOUNT
):
    log.info("Maximum iterations reached without meeting the total amount condition.")
else:
    final_total = allocation_iter["scaled_amount"].sum()
    log.info(
        f"Condition met with {final_total} OP allocated through {current_iteration} iteration(s)."
    )

2024-01-02 19:52:36 INFO | Condition met with 30000000.0 OP allocated through 1 iteration(s).


In [14]:
# join the initial allocation with the final allocation, if scaled_amount is null then make it 0
final_allocation = initial_allocation.merge(
    allocation_iter["scaled_amount"],
    how="left",
    on="project_id",
).fillna({"scaled_amount": 0})

# check if the final allocation table still contains all projects.
if final_allocation.index.nunique() == result_df["project_id"].nunique():
    log.info("Check - Final allocation table has included all the projects.")
else:
    log.info(
        "Check - Final allocation table has missing projects. Printing out the missing projects below."
    )
    log.info(
        result_df[~result_df["project_id"].isin(final_allocation.index)]["project_id"]
    )

# check if the final allocation table still sums to the total amount.
if final_allocation["scaled_amount"].sum() == final_total:
    log.info(
        "Check - Final allocation table sums to the right amount of OP: "
        + str(final_total)
    )
else:
    log.info(
        "Check - Final allocation table does not sum to the total OP. Printing out the missing amount below."
    )
    log.info(str(final_total - final_allocation["scaled_amount"].sum()) + " OP")

2024-01-02 19:52:36 INFO | Check - Final allocation table has included all the projects.
2024-01-02 19:52:36 INFO | Check - Final allocation table sums to the right amount of OP: 30000000.0


In [15]:
# export csv
# allocation_iter.drop(columns="median_amount", inplace=True)
final_allocation.to_csv("data/rpgf3_allocation_final.csv")

log.info(f"Results saved in data/rpgf3_allocation_final.csv.")

2024-01-02 19:52:36 INFO | Results saved in data/rpgf3_allocation_final.csv.


In [16]:
final_allocation.head(10)

,votes_count,median_amount,is_eligible,scaled_amount
project_id,,,,
proj26,5,841223.00,False,0.00
proj323,11,838886.00,False,0.00
proj123,7,829909.00,False,0.00
proj170,10,817377.00,False,0.00
proj46,11,802627.00,False,0.00
proj269,10,793657.00,False,0.00
proj12,11,787796.00,False,0.00
proj196,7,782668.00,False,0.00
proj287,6,782357.00,False,0.00


In [17]:
to_cut = (
    allocation_iter[allocation_iter["scaled_amount"] < 1500]
    .sort_values(by="scaled_amount")
    .head(1)
)

In [18]:
# check if to_cut is empty
to_cut.empty

True

### Calculate Voting Results using pytorch

In [19]:

result_tensor, num_projects = allocator.convert_df_to_tensor(result_df)
project_tensors = allocator.get_project_tensor(result_tensor, num_projects)


In [20]:
# export to onnx
allocator.eval()

# convert projects tensors to tuple
dummy_data = tuple(torch.randn(*x.shape) for x in project_tensors)
initial_allocation = allocator.forward(*project_tensors)
initial_allocation.shape

input_names = ['input_' + str(i) for i in range(len(project_tensors))]
    # Export the model
torch.onnx.export(allocator,               # model being run
                      dummy_data,          # model input (or a tuple for multiple inputs)
                      "network.onnx",           # where to save the model (can be a file or file-like object)
                      export_params=False,       # store the trained parameter weights inside the model file
                      opset_version=17,         # the ONNX version to export the model to
                      do_constant_folding=True, # whether to execute constant folding for optimization
                      input_names = input_names,   # the model's input names
                      output_names = ['output'])

/Users/alexandercamuto/Documents/GitHub/zk-rpgf/core/utils.py:158: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  votes_count = torch.tensor([count]).reshape(1, 1)
/Users/alexandercamuto/Documents/GitHub/zk-rpgf/.env/lib/python3.9/site-packages/torch/onnx/symbolic_opset9.py:2174: FutureWarning: 'torch.onnx.symbolic_opset9._cast_Bool' is deprecated in version 2.0 and will be removed in the future. Please Avoid using this function and create a Cast node instead.
  return fn(g, to_cast_func(g, input, False), to_cast_func(g, other, False))


In [21]:
import ezkl

model_path = "network.onnx"
settings_path = "settings.json"


py_run_args = ezkl.PyRunArgs()
py_run_args.input_visibility = "public"
py_run_args.output_visibility = "public"
py_run_args.param_visibility = "fixed" # private by default

res = ezkl.gen_settings(model_path, settings_path, py_run_args=py_run_args)
assert res == True

Unknown op: ScatterNd
unsupported operation in graph


RuntimeError: Failed to generate settings: General synthesis error